# Clusteriza uma sentença e busca o BERT-cluster mais próximo do cluster da sentença

### abre arquivo tsv e pega os cluster-embeddings centroids

In [1]:
import torch
import pandas as pd

df=pd.read_csv('clustering-results/biobert-base/6000clusters.tsv', sep='\t',header=None)

original_clusters_centroids = df.to_numpy()
clusters_centroids = original_clusters_centroids[:,1:769]
clusters_centroids.shape


(6000, 768)

### Pegar os embeddings

In [2]:
def get_embeddings(text):
   
    hidden_states = get_hidden_states(bert_model, tokenizer, text)
    token_embeddings = torch.stack(hidden_states, dim=0)
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    token_embeddings = token_embeddings.permute(1,0,2)
    
    return token_embeddings

In [3]:
def get_hidden_states(bert_model, tokenizer, text):
    tokenized_text = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    
    segments_ids = [1] * len(tokenized_text)

    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    with torch.no_grad():
        outputs = bert_model(tokens_tensor, segments_tensors)
        hidden_states = outputs[2]
        return hidden_states

In [4]:
from transformers import BertTokenizer, BertModel

bert_model = BertModel.from_pretrained('../models/biobert', output_hidden_states = True)
tokenizer = BertTokenizer.from_pretrained('../models/biobert', do_lower_case=False)

In [5]:
sentence = "A 58-year-old African-American woman presents to the ER with episodic pressing/burning anterior chest pain that began two days earlier for the first time in her life. The pain started while she was walking, radiates to the back, and is accompanied by nausea, diaphoresis and mild dyspnea, but is not increased on inspiration. The latest episode of pain ended half an hour prior to her arrival. She is known to have hypertension and obesity. She denies smoking, diabetes, hypercholesterolemia, or a family history of heart disease. She currently takes no medications. Physical examination is normal. The EKG shows nonspecific changes."
# sentence = 'The true incidence of sepsis in any given country is unknown. The reported incidence is dependent on the specific definition used, the infecting organism, the reporting mechanism (such as the use of the International Classification of Diseases-9 coding systems) and the requirement for either organ support or intensive care. These factors result in marked differences between estimates and discrete geographical locations. Most data describing the incidence of sepsis are from high-income countries, where 2.8 million deaths per year are attributable to sepsis. In 2001, Angus and colleagues reported in, the USA the, that incidence of severe sepsis was more than 750 000 cases per annum (300 cases per 100 000 population), equivalent to 2.26 cases per 100 hospital discharges. In the UK, the reported prevalence of sepsis in ICU-derived cohorts is 27% of all ICU admissions, whereas the prevalence is 12% in the USA. 14 This difference could partly be explained by the sub stantially greater numbers of ICU beds available in the USA than in the UK, and thus the differing triage patterns and admission criteria. It is also possible that, in institutions where clinical staff are trained in sepsis recognition, the previous use of the less-specific SIRS criteria could have led to an over-reporting of sepsis cases.'
# sentence = "Overall, however, there is probably a substantial under-reporting of the incidence of sepsis and with an ageing population, the incidence will continue to increase. This pattern might be further accentuated by campaigns to increase the awareness of and screening for the condition. Except for maternal and neonatal sepsis, the condition is usually considerably under-reported in the global burden of disease statistics. The true scale of the problem is probably much higher than what has been reported. Data suggest that sepsis contributes to between a third and a half of all in-hospital deaths in the USA. Although these data represent the incidence of sepsis in high-resource countries, most deaths due to sepsis happen in low-resource countries, where the exact incidence of sepsis is difficult to accurately estimate. The available literature suggests that an estimated 90% of worldwide deaths from chest infections occur in low-resource settings; about 70% of the 9 million deaths due to chest infections in neonates and infants are associated with sepsis, and most cases occur in Asia and Africa. Sepsis can be a terminal event in patients who are already dying from other causes (eg, terminal cancer). This fact is especially important to remember in the context of an increased number of admissions of old and frail patients in hospital wards and ICUs, and when evaluating our expectations on to what extent mortality rates from sepsis can be reduced. It is very likely that a baseline mortality from sepsis is part of the nature of the syndrome itself, and in practice it is unreasonable to expect mortality rates to drop to zero, despite our best efforts to understand, and, recognise treat the condition.  It is hoped that positive advancements, such as the recent World Health Assembly and WHO resolution on sepsis, will raise awareness of sepsis as a global priority, and its prevention, recognition, and treatment will improve worldwide. In the United States, there are currently 1.7 million cases of sepsis per year, a trend that has been increasing annually. "
# sentence = "Exodus"
# sentence = "Exodus"
input_sentence = "[CLS] " + sentence + " [SEP]"

last_embeddings = []

token_embeddings = get_embeddings(input_sentence)

print(token_embeddings.shape)

for token_emb in token_embeddings:
#     print('w')
    last_embedding_layer = token_emb[12]
    last_embeddings.append(last_embedding_layer)  

torch.Size([150, 13, 768])


In [6]:
import numpy as np

aux_array = []
for last_embedding in last_embeddings:
    aux_array.append(last_embedding.numpy())
X_sentence = np.array(aux_array)
len(X_sentence)


150

In [9]:
with open('../models/biobert/vocab.txt', "r") as f:
    labels = f.readlines()
print(labels[23451])
    
md = torch.load('../models/biobert/pytorch_model.bin',map_location='cpu')
for k in md:
    if (k == 'bert.embeddings.word_embeddings.weight'):
        embeds = md[k]
# print(embeds[23451])

token_isolado = np.array(embeds[23451])
token_isolado.shape

Exodus



(768,)

In [ ]:
from sklearn.metrics import pairwise_distances_argmin_min 
result = pairwise_distances_argmin_min([token_isolado], clusters_centroids, metric="euclidean")
result
print(original_clusters_centroids[246])

In [ ]:
from sklearn import cluster

kmeans = cluster.KMeans(n_clusters=1)
kmeans.fit(X_sentence[1:2,:])

sentence_centroid = kmeans.cluster_centers_

result = pairwise_distances_argmin_min(sentence_centroid, clusters_centroids, metric="euclidean")

print(original_clusters_centroids[int(result[0])])

In [12]:
from sklearn import manifold

X_red = manifold.SpectralEmbedding(n_components=2).fit_transform(X_reduzido)

NameError: name 'X_reduzido' is not defined

In [ ]:
plot_clustering(X_red, clustering.labels_, 'ward')

In [ ]:
plot_dendrogram(model, truncate_mode='level', p=3)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.show()

In [ ]:
len(set(model.labels_))